In [159]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

import os

import numpy as np

In [160]:
def preprocess_vgg16_input(image):
    # Convert image to numpy array and transpose dimensions
    image = np.array(image)
    image = image.transpose((2, 0, 1))  # Convert from HWC to CHW format

    # Convert image to torch tensor
    image = torch.tensor(image).float()

    # Preprocess using mean subtraction
    mean = torch.tensor([0.485, 0.456, 0.406]).reshape(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).reshape(3, 1, 1)
    image = (image / 255.0 - mean) / std

    return image

In [161]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.Lambda(lambda x: preprocess_vgg16_input(x)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # transforms.ToTensor()   # Convert images to tensors
])

In [162]:
# training and testing directories

def create_folder(x):
    if os.path.exists(x):
        print("Folder already exists !!!")

    else:
        os.mkdir(x)
        print("{x} created Successfully !!!")


folderpath_Train = r"D:\vs code\python\DeepLearning\Projects\Tumor\train"
folderpath_Test = r"D:\vs code\python\DeepLearning\Projects\Tumor\test"
folderpath_Validate = r"D:\vs code\python\DeepLearning\Projects\Tumor\validate"

create_folder(folderpath_Train)
create_folder(folderpath_Test)
create_folder(folderpath_Validate)


folder_path_train_damaged = r"D:\vs code\python\DeepLearning\Projects\Tumor\train\damaged"
folder_path_train_good = r"D:\vs code\python\DeepLearning\Projects\Tumor\train\good"
folder_path_test_damaged = r"D:\vs code\python\DeepLearning\Projects\Tumor\test\damaged"
folder_path_test_good = r"D:\vs code\python\DeepLearning\Projects\Tumor\test\good"
folder_path_validate_damaged = r"D:\vs code\python\DeepLearning\Projects\Tumor\validate\damaged"
folder_path_validate_good = r"D:\vs code\python\DeepLearning\Projects\Tumor\validate\good"


create_folder(folder_path_train_damaged)
create_folder(folder_path_train_good)

create_folder(folder_path_test_damaged)
create_folder(folder_path_test_good)

create_folder(folder_path_validate_good)
create_folder(folder_path_validate_damaged)


Folder already exists !!!
Folder already exists !!!
Folder already exists !!!
Folder already exists !!!
Folder already exists !!!
Folder already exists !!!
Folder already exists !!!
Folder already exists !!!
Folder already exists !!!


In [163]:
train_dataset = datasets.ImageFolder(root=folderpath_Train,transform=transform)

test_dataset = datasets.ImageFolder(root=folderpath_Test,transform=transform)

validation_dataset = datasets.ImageFolder(root=folderpath_Validate,transform=transform)


In [164]:
train_loader = DataLoader(train_dataset,batch_size=32, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size= 32, shuffle= False)

validate_loader = DataLoader(validation_dataset, batch_size=32, shuffle= False)

In [165]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x
    

model = SimpleCNN()

In [166]:
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


Initializing optimizer and loss function

In [167]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [168]:
# Training Loop

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad() # Zero the parameter gradients
        output = model(inputs)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step() # # Optimize

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss/ len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}")

    # Validation loop
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    val_corrects = 0

    with torch.no_grad():
        for inputs, labels in validate_loader:
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            val_running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)

    # Calculate average validation loss and accuracy for the epoch
    val_epoch_loss = val_running_loss / len(validate_loader.dataset)
    val_accuracy = val_corrects.double() / len(validate_loader.dataset)
    print(f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_accuracy:.4f}")


# Testing loop
model.eval()  # Set model to evaluation mode
test_corrects = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)  # Forward pass
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data)

# Calculate accuracy on test set
test_accuracy = test_corrects.double() / len(test_loader.dataset)
print(f"Test Accuracy: {test_accuracy:.4f}")


Epoch 1/10, Training Loss: 0.6445
Validation Loss: 0.5673, Accuracy: 0.7593
Epoch 2/10, Training Loss: 0.3276
Validation Loss: 0.2726, Accuracy: 0.8956
Epoch 3/10, Training Loss: 0.2370
Validation Loss: 0.2127, Accuracy: 0.9133
Epoch 4/10, Training Loss: 0.1692
Validation Loss: 0.1666, Accuracy: 0.9345
Epoch 5/10, Training Loss: 0.1443
Validation Loss: 0.1520, Accuracy: 0.9310
Epoch 6/10, Training Loss: 0.0769
Validation Loss: 0.1366, Accuracy: 0.9434
Epoch 7/10, Training Loss: 0.0622
Validation Loss: 0.1567, Accuracy: 0.9575
Epoch 8/10, Training Loss: 0.0572
Validation Loss: 0.1137, Accuracy: 0.9681
Epoch 9/10, Training Loss: 0.0279
Validation Loss: 0.1251, Accuracy: 0.9593
Epoch 10/10, Training Loss: 0.0115
Validation Loss: 0.1266, Accuracy: 0.9681
Test Accuracy: 0.9699
